### Fitting Naive Bayes to the Training set

posterior = prior occurrances * liklihood / evidence
The "gaussian" and "naive" come from two assumptions present in this likelihood:
1. we assume each feature is uncorrelated from each other. That is, meannfreq is                  independent of sd or median etc.. This is obviously not true, and is a "naive"                assumption.
2. we assume have that the value of the features (e.g. meannfreq of women) are normally          (gaussian) distributed. This means that p(meannfreq∣female) is calculated by inputing          the required parameters into the probability density function of the normal                    distribution.


In [1]:
from sklearn.naive_bayes import GaussianNB
import pandas as pd 
import numpy as np  
import random
from sklearn.model_selection import train_test_split  
from collections import defaultdict

In [74]:
%store -r train_x
%store -r test_x
%store -r train_y
%store -r test_y
%store -r genderLabels

%store -r train_x_two_features
%store -r test_x_two_features
%store -r train_y_two_features
%store -r test_y_two_features

%store -r transformed_train_x
%store -r transformed_test_x

In [87]:
import pandas as pd
label_map = {
    0: 'female',
    1: 'male'
}
temp_train_y.map(label_map)

AttributeError: 'numpy.ndarray' object has no attribute 'map'

In [86]:
temp_train_y

array([[0],
       [1],
       [0],
       ..., 
       [1],
       [1],
       [1]])

In [29]:
temp_train_y = train_y.reshape(len(train_y), 1)

In [118]:
# female -> 0
# male -> 1
# Translate every element in numpy array according to key
my_dict = {0:"female",1:"male"}
temp_train_y = np.vectorize(my_dict.get)(train_y)
# Reshape train_y
temp_train_y = temp_train_y.reshape(len(temp_train_y), 1)
# Join two arrays vertically
train_df = np.concatenate((train_x, temp_train_y), axis=1)
# Columns
columns_new = ["meanfreq","sd","median","Q25","Q75","IQR","skew","kurt","sp.ent","sfm","mode","centroid","meanfun","minfun","maxfun","meandom","mindom","maxdom","dfrange","modindx","label"]
# Pass in array and columns
train_df =pd.DataFrame(train_df, columns=columns_new)

In [120]:
# For both the classes 0 and 1, and for each of the 20 features, we have thus saved the mean and the standard in summary. 
# For example, for the class 0 (female), and feature 1 (ie, sd), mean and the standard for this attribute are 0.060910268026077803 and 0.00034668419803097784

def getSummary(data,n_cls):
    data_means = data.groupby('label').mean()
    data_variance = data.groupby('label').var()
    
    cnames = np.array(data.columns.tolist())[:-1] 
    classes = np.unique(labels)
    classnames = np.array(['female','male'])
    summary = {}

    for cls in classes:
        #initializing the dictionary
        summary[cls] = defaultdict(list)
    for cls in classes:
        for j in range(0, n_cls):
            print(classnames[cls])
            summary[cls][j] += list(np.array([data_means[cnames[j]][data_means.index == classnames[cls]].values[0],
                                          data_variance[cnames[j]][data_variance.index == classnames[cls]].values[0]]))
            
    print summary   
    return summary

In [122]:
summary = getSummary(train_df,train_x.shape[1])

DataError: No numeric types to aggregate

In [47]:
# Group the data by gender and calculate the means of each feature
data_means = train_x_df.groupby('label').mean()
# View the values
data_means

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
label,,,,,,,,,,,,,,,,,,,,
0.0,0.314653,-0.455157,0.261732,0.484678,-0.070203,-0.590939,-0.031266,-0.084121,-0.471300,-0.337293,0.158686,0.314653,0.811053,0.118176,0.166368,0.189798,0.183518,0.189015,0.185805,-0.021680
1.0,-0.330399,0.477935,-0.274830,-0.508933,0.073716,0.620511,0.032830,0.088331,0.494885,0.354173,-0.166627,-0.330399,-0.851641,-0.124090,-0.174694,-0.199296,-0.192702,-0.198474,-0.195104,0.022765


In [49]:
# Group the data by gender and calculate the variance of each feature
data_variance = train_x_df.groupby('label').var()
# View the values
data_variance

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
label,,,,,,,,,,,,,,,,,,,,
0.0,1.020998,1.223149,0.798805,1.056744,0.973134,1.020283,0.553424,0.552644,1.102338,1.040779,0.772381,1.020998,0.330011,1.295627,0.523229,1.200111,1.299962,1.202630,1.204760,0.835003
1.0,0.766369,0.321059,1.065409,0.436022,1.019320,0.228061,1.468564,1.456243,0.415699,0.713796,1.186494,0.766369,0.288032,0.661205,1.442745,0.713981,0.614181,0.711975,0.712331,1.173976


In [50]:
# Create a function to calculate the probability density of each of the terms of the likelihood 
# We use mean and standard deviation of input values (x) for each class to summarize the distribution
# Probabilities of new x values are calculated using the Gaussian Probability Density Function (PDF).

def p_x_given_y(x, mean_y, variance_y):
    # Input the arguments into a probability density function
    p = 1/(np.sqrt(2*np.pi*variance_y)) * np.exp((-(x-mean_y)**2)/(2*variance_y))
    return p

# We calculate the probability that the value of the random variable will be between x - 0.01 and x + 0.01


In [52]:
# Test
p_x_given_y(2, 2, 4)

0.19947114020071635

In [5]:
def calculateClassProbabilities(summaries, inputVector):
    # p(x|y) (e.g. p(meannfreq∣female))
    probabilities = {}
    for classValue, classSummaries in summaries.iteritems():
        if (classValue == 0):
            probabilities[classValue] = P_female
        
        else:
            probabilities[classValue] = P_male
        for i in range(len(classSummaries)):
            mean, var = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= p_x_given_y(x, mean, var)
    return probabilities

In [ ]:
calculateClassProbabilities(summary, test_x[10])
# PDF can be greater than 1, but it can't be greater than 1 for a large interval
# because of the non-negativity and integral constraints.

In [ ]:
# Make a Prediction
import operator
def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    #print probabilities
    probabilities = sorted(probabilities.iteritems(), key=operator.itemgetter(1), reverse=True)
    return probabilities[0][0]

In [ ]:
print(test_x[10])
print('Our prediction for test_x[10] is %s'  % (predict(summary, test_x[10])))

In [ ]:
from sklearn.metrics import classification_report

def GaussianNB_Classifier(test_x,test_y):
    results = []
    for i in range(len(test_x)):
        test_single = test_x[i]
        res = predict(summary, test_single)
        results.append(res)
    # We evaluate the accuracy of the model by calculating a ratio of the total correct predictions out of all predictions made (the classification accuracy)
    correct = 0
    wrong = 0
    for j in range(len(test_y)):
        if(results[j] == test_y[j]):
            correct += 1
        else:
            wrong += 1
 
    print('Correct rate is %s'  % (float(correct)/(correct+wrong)))
    print(classification_report(test_y,results))



In [ ]:
print("Use classification reports to evaluate how well Gaussian Naive Bayes performed with all features \n")
print("in-sample accuracy in Gaussian Naive Bayes:")
GaussianNB_Classifier(train_x,train_y)

In [ ]:
print("out-of-sample accuracy in Gaussian Naive Bayes:")
GaussianNB_Classifier(test_x,test_y)

In [ ]:
# We are wondering if there is a big difference in the results 
# when used Sklearn package vs. our own algorithm
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

nb = GaussianNB()
nb.fit(train_x, train_y)
prediction = nb.predict(test_x)

print('NB result: ', accuracy_score(test_y, prediction))

In [ ]:
# Is it enough of these two features to make predictions? 
## Random Forest with 2 features ('meanfun', 'IQR')

voice_new_data = voice_data[['IQR','meanfun','label']]
train_new_x = train_x[:,[5,12]]
test_new_x = test_x[:,[5,12]]
summary = getSummary(voice_new_data, train_new_x.shape[1])

In [ ]:
print("Use classification reports to evaluate how well Gaussian Naive Bayes performed with only 2 features \n")
print("In-sample accuracy in Gaussian Naive Bayes:")
GaussianNB_Classifier(train_new_x,train_y)

In [ ]:
print("Out-of-sample accuracy in Gaussian Naive Bayes:")
GaussianNB_Classifier(test_new_x,test_y)

We did observe some improvement in performance when we implement Gaussian Naive Bayes with 2 features.